In [36]:
import pandas as pd

# Reads the data
emissions = pd.read_csv('co2-fossil-by-nation_zip/data/fossil-fuel-co2-emissions-by-nation_csv.csv')
emissions['Country'] = emissions['Country'].map(lambda n: n.lower())
print(emissions.shape)

# Deletes extra unnamed columns with NaNs
population = pd.read_csv('API_SP.POP.TOTL_DS2_en_csv_v2_4751604/API_SP.POP.TOTL_DS2_en_csv_v2_4751604.csv').dropna(how='all', axis='columns')
print(population.shape)
population['Country Name'] = population['Country Name'].map(lambda n: n.lower())

gdps = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4751562/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4751562.csv').dropna(how='all', axis='columns')
gdps['Country Name'] = gdps['Country Name'].map(lambda n: n.lower())
print(gdps.shape)


(17232, 10)
(266, 66)
(266, 66)


In [37]:
# Prepares for merge of data from all datasources
import numpy as np
import difflib

countries_emissions = np.sort(emissions['Country'].unique())
countries_list = np.sort(population['Country Name'].unique())
# GDP and Population data uses the same encoding

decapitalise = np.vectorize(lambda x : x.lower())
countries_emissions = decapitalise(countries_emissions)
countries_list = decapitalise(countries_list)

matches = {} # name -> corresponding name in the second dataframe

for name in countries_emissions:
    res = difflib.get_close_matches(name, countries_list, len(countries_list), 0)[0]
    matches[name] = res

# Manual updates
matches['czech republic'] = 'czechia'
matches['china (mainland)'] = 'china'
matches['democratic republic of the congo (formerly zaire)'] = 'congo, dem. rep.'
matches['congo'] = 'congo, rep.'
matches['democratic republic of vietnam'] = 'vietnam'
matches['federated states of micronesia'] = 'micronesia, fed. sts.'
matches['gambia'] = 'gambia, the'
matches['france (including monaco)'] = 'france'
matches['kyrgyzstan'] = 'kyrgyz republic'
matches['federal republic of germany'] = 'germany'
matches['japan (excluding the ruyuku islands)'] = 'japan'
matches['lao people s democratic republic'] = 'lao pdr'
matches['italy (including san marino)'] = 'italy'
matches['islamic republic of iran'] = 'iran, islamic rep.'
matches['plurinational state of bolivia'] = 'bolivia'
matches['slovakia'] = 'slovak republic'
matches['myanmar (formerly burma)'] = 'myanmar'
matches['republic of korea'] = 'korea, rep.'
matches['libyan arab jamahiriyah'] = 'libya'

# Corrupting inputs
del matches['zanzibar']
del matches['antarctic fisheries']
del matches['christmas island']
del matches['ussr']
del matches['united korea']
del matches['former panama canal zone']
del matches['occupied palestinian territory']
del matches['saint helena']
del matches['republic of south vietnam']
del matches['taiwan']
del matches['yugoslavia (former socialist federal republic)']
del matches['bonaire, saint eustatius, and saba']
del matches['niue']
del matches['st. pierre & miquelon']

emissions[emissions['Country'] == 'czech republic']

,Year,Country,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total)
12315,1992,czech republic,37681,27799,5443,3622,816,0,3.65,148
12528,1993,czech republic,36169,26493,5241,3702,733,0,3.50,121
12741,1994,czech republic,34288,24708,5243,3616,721,0,3.32,148
12955,1995,czech republic,33769,24164,4857,4092,656,0,3.27,157
13169,1996,czech republic,34191,23770,5030,4709,682,0,3.32,114
13383,1997,czech republic,34746,24315,4978,4790,663,0,3.37,105
13597,1998,czech republic,33418,22622,5370,4800,626,0,3.25,98
13811,1999,czech republic,30536,19930,5203,4826,577,0,2.98,114
14025,2000,czech republic,33774,23581,4951,4684,557,0,3.30,134
14239,2001,czech republic,33739,22968,5271,5017,483,0,3.30,135


In [38]:
# Deletes 3% of inputs from Emission that were agreed to be corrupted
emissions = emissions[emissions['Country'].isin(matches)] 

# Maps the names of countries to match
emissions['Country'] = emissions['Country'].map(matches)
emissions.head()

,Year,Country,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total)
0,1751,united kingdom,2552,2552,0,0,0,0,0.0,0
1,1752,united kingdom,2553,2553,0,0,0,0,0.0,0
2,1753,united kingdom,2553,2553,0,0,0,0,0.0,0
3,1754,united kingdom,2554,2554,0,0,0,0,0.0,0
4,1755,united kingdom,2555,2555,0,0,0,0,0.0,0


In [39]:
# Combine emssions with gdp and population data
data = emissions
data['Population'] = np.nan
data['GDP'] = np.nan

def getPopulation(country_name, year):
    try:
        return np.float64(population.loc[(population['Country Name'] == country_name)][str(year)])
    except:
        return np.nan

def getGDP(country_name, year):
    try:
        return np.float64(gdps.loc[(population['Country Name'] == country_name)][str(year)])
    except:
        return np.nan 
    
def parsePopulation(row):
    row['Population'] = getPopulation(row['Country'], row['Year'])
    row['GDP'] = getGDP(row['Country'], row['Year'])
    return row

data = data.apply(parsePopulation, axis=1)

data.shape

(16567, 12)

In [7]:
data[np.isnan(data['Population']) == False] # Population data not available for 36% of emission inputs
data[np.isnan(data['GDP']) == False] # GDP data not available for 46% of emission inputs

,Year,Country,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total),Population,GDP
6352,1960,afghanistan,113,35,74,0,5,0,0.01,0,8996967.0,5.377778e+08
6354,1960,algeria,1680,343,1193,0,144,0,0.16,438,11057864.0,2.723593e+09
6358,1960,australia,24053,16083,7590,0,380,0,2.34,1219,10276477.0,1.860679e+10
6359,1960,austria,8405,4926,2297,797,385,0,1.19,1,7047539.0,6.592694e+09
6360,1960,"bahamas, the",112,0,112,0,0,0,1.02,468,109532.0,1.698039e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
17227,2014,vietnam,45517,19246,12694,5349,8229,0,0.49,761,91713850.0,2.334500e+11
17228,2014,turks and caicos islands,6,0,6,0,0,0,0.44,1,35371.0,8.410700e+08
17229,2014,"yemen, rep.",6190,137,5090,581,381,0,0.24,153,25823488.0,4.322859e+10
17230,2014,zambia,1228,132,797,0,299,0,0.08,33,15399793.0,2.714102e+10


In [13]:
# Które kraje w poszczególnych latach z danymi, emitują najwięcej CO2 w
# przeliczeniu na mieszkańca. To znaczy generuje posortowaną po latach
# tabelkę pięcioma krajami o największej emisji na osobę (z podaną nazwą
# kraju, emisją na osobę i całkowitą emisją

#[(data['Year'] => year_range_start) & (data['Year'] <= year_range_end)]

r = data[['Year', 'Country', 'Per Capita', 'Total']][(data['Year'] > 1980) & (data['Year'] <= 2005)].sort_values('Per Capita').groupby(['Year']).tail(5).sort_values(['Year', 'Per Capita'], ascending=False)
r.head(12)


,Year,Country,Per Capita,Total
15208,2005,qatar,16.93,13896
15156,2005,kuwait,8.62,19511
15187,2005,netherlands,8.41,1564
15247,2005,trinidad and tobago,7.91,10404
15255,2005,united arab emirates,7.78,31674
14992,2004,qatar,16.38,11711
14971,2004,netherlands,8.60,1578
15039,2004,united arab emirates,8.44,30881
14940,2004,kuwait,7.91,17326
14847,2004,aruba,7.21,714


In [15]:
# Które kraje w poszczególnych latach z danymi mają największy przychód
# mieszkańca. To znaczy generuje posortowaną po latach tabelkę pięcioma
# krajami o największym dochodzie na mieszkańca (z podaną nazwą kraju,
# dochodem na mieszkańca i całkowitym dochodem)

data['GDP per capita'] = data['GDP'] / data['Population']
data[['Year', 'Country', 'GDP per capita', 'GDP']][(data['Year'] >= 1960) & (data['Year'] <= 2005)].dropna().sort_values('GDP per capita').groupby(['Year']).tail(5).sort_values(['Year', 'GDP per capita'], ascending=False)

,Year,Country,GDP per capita,GDP
15165,2005,luxembourg,80988.137623,3.767228e+10
15075,2005,bermuda,75882.033856,4.868136e+09
15195,2005,norway,66810.478521,3.088843e+11
15142,2005,iceland,56794.850159,1.685296e+10
15239,2005,switzerland,56546.785655,4.205449e+11
...,...,...,...,...
6520,1960,united states,3007.123445,5.433000e+11
6462,1960,new zealand,2312.949992,5.485855e+09
6372,1960,canada,2259.294285,4.046172e+10
6445,1960,luxembourg,2242.015817,7.039257e+08


In [46]:
# Które kraje (w przeliczeniu na mieszkańca) najbardziej zmniejszyły i
# zwiększyły przez ost. 10 lat (z danych) emisję CO2

year = data.dropna()['Year'].max()
year_back = year - 10
data_ch = data[data['Year'] == year][['Country', 'Per Capita']].merge(data[data['Year'] == year_back][['Country', 'Per Capita']], left_on='Country', right_on='Country')
data_ch['Change (10 yrs)'] = data_ch['Per Capita_x'] - data_ch['Per Capita_y']
data_ch.sort_values('Change (10 yrs)').tail(5)[['Country', 'Change (10 yrs)']].iloc[::-1] # Worst (increased emmision)
data_ch.sort_values('Change (10 yrs)', ascending=False).tail(5)[['Country', 'Change (10 yrs)']].iloc[::-1] # Best

 

,Country,Change (10 yrs)
153,netherlands,-5.90
11,aruba,-4.91
172,qatar,-2.84
5,angola,-2.11
218,united arab emirates,-2.10
